In [1]:
import pandas as pd

In [2]:
from pymongo import MongoClient

CONNECTION_STRING = "mongodb://127.0.0.1:27017/isanford"
client = MongoClient(CONNECTION_STRING)
#client = MongoClient('localhost', 27017)

db = client["isanford"]

players = db["players"]
games = db["games"]


In [ ]:
# probably unnecessary? idk. Could work as a last filtery step
alignment = pd.read_csv("data/keys/alignment.csv").reset_index().rename(columns={"index": "alignment_id"})
print(alignment.head())

setting = pd.read_csv("data/keys/setting.csv")
print(setting.head())

death = pd.read_csv("data/keys/death.csv").reset_index().rename(columns={"index": "death_id"})
death.head()

In [4]:
# IMPORTING DATA TAB
data = pd.read_csv("data/original_data/orig_data_040622.csv")

data.rename(inplace=True, columns={
    "ID": "pg_id",
    "Player": "player_name",
    "Alignment": "alignment_char",
    "Faction Outcome": "win", 
    "First Hit": "first_hit", 
    "Last Hit": "last_hit",
    "# of Hits": "num_hits",
    "Death/\r\n Survival": "death_char", # ??? sometimes \r\n sometimes \n
    "Inactive": "inactive",
    "Mod": "IM",
    "Game Type": "game_format",
    "Game #": "game_num",
    "Unnamed: 15": "game_string", # these depend on other cols. name in sheet pls
    "Role": "role",
    "Unnamed: 18": "secondary_role",
    "Main?": "main_gm"
    })

print(data.columns)

data = data[data["Spec"] != "Y"]

data = data.filter([
    "pg_id", 
    "player_name",
    "alignment_char",
    "win",
    "first_hit", "last_hit", "num_hits",
    "death_char",
    "inactive",
    "GM", "main_gm", "IM", # "Spec",
    "game_string", #"game_format", "game_num",
    "role", "secondary_role"
    ])

data.head()

# more cleaning here? 

Index(['pg_id', 'player_name', 'alignment_char', 'win', 'first_hit',
       'last_hit', 'num_hits', 'death_char', 'inactive', 'GM', 'main_gm', 'IM',
       'Spec', 'game_format', 'game_num', 'game_string', 'Broken', 'role',
       'secondary_role', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')


,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG1,GM,Roles that are unique and don't fit
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG1,Tineye,MR1/LG8/MR7 need work
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Cop,NaN
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Regular,NaN
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG1,Smoker,NaN


In [5]:
data.tail()

,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role
5890,5891,StrikerEZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR57,NaN,NaN
5891,5892,Bort,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR57,NaN,NaN
5892,5893,TJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR57,NaN,NaN
5893,5894,Ashbringer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG84,GM,NaN
5894,5895,Elbereth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,LG84,IM,NaN


In [6]:
# IMPORTING GAME TAB

# header = 1 because there's an extra header row on top w categories
game = pd.read_csv("data/original_data/orig_game_040622.csv", header=1)

game.reset_index(inplace=True)

game.columns

Index(['index', 'format', 'num', 'auto?', '# Players', '# Cycles', '# Posts',
       'Fundamentals', 'Winner', 'Complexity', 'Role Madness', 'Anon', 'Mech',
       'Dist', 'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'Title/Link', 'Sanderson? ', 'World', 'start_date',
       'end_date', 'link'],
      dtype='object')

In [7]:

# ! Need to be careful about sheet renaming cols - some kind of check?
game.rename(inplace=True, columns={
    "index": "game_id",
    "format": "game_format",
    "num": "game_num", 
    "auto?": "game_string", # optional? 
    "# Players": "num_players", # optional
    "# Cycles": "num_cycles",
    "# Posts": "num_posts",
    "Fundamentals": "fundamentals", 
    "Role Madness": "role_madness",
    "Winner": "winner", # optional
    "Complexity": "complexity",
    #"Broken": "broken", # TODO make into mech/dist once that's set in sheet
    "Mech": "mechanics_balance", # blank rn!!
    "Dist": "distribution_balance",
    "World": "setting", # need to make setting table and replace this w setting_id
    "Anon": "anon_num", 
    "Title/Link": "title", # need to get link separately. hidden col? 
    # LINK
    "start_date": "start_date",
    "end_date": "end_date",
    "# Elims": "num_elims",
    "# Survivors": "num_survivors",
    "Sanderson?": "is_sanderson"
    })

print(game.columns)

# remaining columns are just calculated from data test
game = game.filter([
    "game_id",
    "game_format", "game_num", "game_string",
    "num_players",
    #"IM", # currently blank - pulling from Data instead
    "num_cycles", 
    "num_posts", 
    "fundamentals", "role_madness", "complexity", 
    "winner", 
    "mechanics_balance", "distribution_balance",
    #"broken", 
    "setting",
    "anon_num",
    "title",
    "start_date", "end_date",
    "num_elims",
    "num_survivors",
])


game.tail()
# check for almost all NaN rows! extra checkboxes cause this

Index(['game_id', 'game_format', 'game_num', 'game_string', 'num_players',
       'num_cycles', 'num_posts', 'fundamentals', 'winner', 'complexity',
       'role_madness', 'anon_num', 'mechanics_balance', 'distribution_balance',
       'Broken', 'num_elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       'num_survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'title', 'Sanderson? ', 'setting', 'start_date',
       'end_date', 'link'],
      dtype='object')


,game_id,game_format,game_num,game_string,num_players,num_cycles,num_posts,fundamentals,role_madness,complexity,winner,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date,num_elims,num_survivors
208,208,LG,83.0,LG83,28.0,6,NaN,V/E,False,NaN,E,NaN,NaN,NaN,NaN,The Survivor,NaN,NaN,6.0,7.0
209,209,BT,1.0,BT1,12.0,4,NaN,V/E,False,NaN,E,NaN,NaN,Cross-Cosmere,NaN,Auction of Stories,NaN,NaN,NaN,NaN
210,210,QF,59.0,QF59,15.0,7,NaN,V/E,False,NaN,E,NaN,NaN,Roshar,NaN,Bachelor - Roshar Edition!,NaN,NaN,NaN,NaN
211,211,MR,57.0,MR57,NaN,NaN,NaN,V/E,False,Semi Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,212,LG,84.0,LG84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
games_db = {}

for full_row in game.iterrows():
    row = full_row[1]

    game_dict = {
        "_id": row["game_id"],
        "format": row["game_format"],
        "game_num": row["game_num"],
        "game_string": row["game_string"],

        "basics": {
            "fundamentals": row["fundamentals"],
            "complexity": row["complexity"],
            "role_madness": row["role_madness"]
        },

        "player_stats": {
            "num_players": row["num_players"],
            "num_elims": row["num_elims"],
            "num_survivors": row["num_survivors"],
            "player_list": []
        },

        "winner": row["winner"],
        
        "num_cycles": row["num_cycles"],
        "num_posts": row["num_posts"],
        "title": row["title"],

        "balance": {
            "mechanics": row["mechanics_balance"],
            "distribution": row["distribution_balance"]
        },

        "setting": { # idk
            "world": row["setting"]
            #"sanderson": row["is_sanderson"]
        },

        "GMs": []

        # want start and end date once present
    }

    if str(row["anon_num"]) != "nan": # gotta be a better way
        game_dict["anon_num"] = row["anon_num"]
    
    games_db[row["game_string"]] = game_dict


# still want: IM, GMs, playerlist (at least), is sanderson? 


In [9]:
games_db["MR38.1"]

{'_id': 145,
 'format': 'MR',
 'game_num': 38.1,
 'game_string': 'MR38.1',
 'basics': {'fundamentals': 'U', 'complexity': 'Basic', 'role_madness': False},
 'player_stats': {'num_players': 18.0,
  'num_elims': 5.0,
  'num_survivors': 15.0,
  'player_list': []},
 'winner': 'E',
 'num_cycles': nan,
 'num_posts': nan,
 'title': 'The Council of Elrond',
 'balance': {'mechanics': nan, 'distribution': nan},
 'setting': {'world': 'Lord of the Rings'},
 'GMs': []}

In [11]:
players_db = {}

# maybe make player ids

for full_row in data.iterrows():
    row = full_row[1]

    name = row["player_name"]
    if name not in players_db:
         
        player_dict = {
            "player_name": name,
            "games": [],
            # other stats? 
        }

        players_db[name] = player_dict
    
    p = players_db[name]

    # spectator check???

    if (row["GM"] == "Y"):
        if "GMed" not in p:
            p["GMed"] = []
        
        p["GMed"].append({
            "game_string": row["game_string"],
            "main": True if row["main_gm"] == "Y" else False,
            # get other stuff ... somehow? idk 
        })

        games_db[row["game_string"]]["GMs"].append(name)

        continue # ehhhh
    
    if (row["IM"] == "Y"):
        if "IMed" not in p:
            p["IMed"] = []
        
        p["IMed"].append({
            "game_string": row["game_string"]
            # ????
        })

        games_db[row["game_string"]]["IM"] = name

        continue
    
    p["games"].append({
        "game_string": row["game_string"],
        "alignment": row["alignment_char"],
        "death": row["death_char"],
        "result": row["win"],
        "hits": {
            "first": row["first_hit"],
            "last": row["last_hit"],
            "num": row["num_hits"]
        },
        "roles": [row["role"], row["secondary_role"]],
        "inactive": True if row["inactive"] == "Y" else False,
    })

    try:     
        games_db[row["game_string"]]["player_stats"]["player_list"].append(name)
    except:
        print(row)
        break
    



In [12]:
games_db["MR38.1"]
# why do I show up as GMing twice??

{'_id': 145,
 'format': 'MR',
 'game_num': 38.1,
 'game_string': 'MR38.1',
 'basics': {'fundamentals': 'U', 'complexity': 'Basic', 'role_madness': False},
 'player_stats': {'num_players': 18.0,
  'num_elims': 5.0,
  'num_survivors': 15.0,
  'player_list': ['Haelbarde',
   'Straw',
   'Fifth Scholar',
   'Coda',
   'Kasimir',
   'Elandera',
   'Amanuensis',
   'xinoehp512',
   'Wonko the Sane',
   'Pejidot',
   'Burnt Spaghetti',
   'Aragorn',
   'StrikerEZ',
   'Madagascar',
   'GoWibble',
   'Rathmaskal',
   'STINK',
   'Young Bard',
   'Haelbarde',
   'Straw',
   'Fifth Scholar',
   'Coda',
   'Kasimir',
   'Elandera',
   'Amanuensis',
   'xinoehp512',
   'Wonko the Sane',
   'Pejidot',
   'Burnt Spaghetti',
   'Aragorn',
   'StrikerEZ',
   'Madagascar',
   'GoWibble',
   'Rathmaskal',
   'STINK',
   'Young Bard']},
 'winner': 'E',
 'num_cycles': nan,
 'num_posts': nan,
 'title': 'The Council of Elrond',
 'balance': {'mechanics': nan, 'distribution': nan},
 'setting': {'world': 'Lord

In [13]:

players_db["Elbereth"]

{'player_name': 'Elbereth',
 'games': [{'game_string': 'LG15.1',
   'alignment': 'G',
   'death': 'S',
   'result': 'L',
   'hits': {'first': nan, 'last': nan, 'num': 0.0},
   'roles': [nan, nan],
   'inactive': False},
  {'game_string': 'LG15.2',
   'alignment': 'G',
   'death': 'S',
   'result': 'W',
   'hits': {'first': nan, 'last': nan, 'num': 0.0},
   'roles': ['Kill', nan],
   'inactive': False},
  {'game_string': 'MR10',
   'alignment': 'G',
   'death': 'S',
   'result': 'L',
   'hits': {'first': nan, 'last': nan, 'num': 0.0},
   'roles': [nan, nan],
   'inactive': False},
  {'game_string': 'AG2',
   'alignment': 'G',
   'death': 'S',
   'result': 'L',
   'hits': {'first': nan, 'last': nan, 'num': 0.0},
   'roles': [nan, nan],
   'inactive': False},
  {'game_string': 'QF12',
   'alignment': 'G',
   'death': 'E',
   'result': 'L',
   'hits': {'first': nan, 'last': nan, 'num': 1.0},
   'roles': [nan, nan],
   'inactive': False},
  {'game_string': 'LG16',
   'alignment': 'G',
   'd

In [14]:
resP = players.insert_many(players_db.values())

resP


In [15]:
resG = games.insert_many(games_db.values())

resG

# indices

db.players.createindex({"player_name": 1})

db.games.createindex({"game_format": 1})

In [17]:
resP.inserted_ids

[ObjectId('624e5632d20eaa6182dc3889'),
 ObjectId('624e5632d20eaa6182dc388a'),
 ObjectId('624e5632d20eaa6182dc388b'),
 ObjectId('624e5632d20eaa6182dc388c'),
 ObjectId('624e5632d20eaa6182dc388d'),
 ObjectId('624e5632d20eaa6182dc388e'),
 ObjectId('624e5632d20eaa6182dc388f'),
 ObjectId('624e5632d20eaa6182dc3890'),
 ObjectId('624e5632d20eaa6182dc3891'),
 ObjectId('624e5632d20eaa6182dc3892'),
 ObjectId('624e5632d20eaa6182dc3893'),
 ObjectId('624e5632d20eaa6182dc3894'),
 ObjectId('624e5632d20eaa6182dc3895'),
 ObjectId('624e5632d20eaa6182dc3896'),
 ObjectId('624e5632d20eaa6182dc3897'),
 ObjectId('624e5632d20eaa6182dc3898'),
 ObjectId('624e5632d20eaa6182dc3899'),
 ObjectId('624e5632d20eaa6182dc389a'),
 ObjectId('624e5632d20eaa6182dc389b'),
 ObjectId('624e5632d20eaa6182dc389c'),
 ObjectId('624e5632d20eaa6182dc389d'),
 ObjectId('624e5632d20eaa6182dc389e'),
 ObjectId('624e5632d20eaa6182dc389f'),
 ObjectId('624e5632d20eaa6182dc38a0'),
 ObjectId('624e5632d20eaa6182dc38a1'),
 ObjectId('624e5632d20eaa